
# INSTALLATION

In [ ]:
!pip install aif360
!pip install fairlearn

In [ ]:
!apt-get install -jre
!java -version

In [ ]:
!pip install h2o

In [ ]:
!pip install xlsxwriter

In [5]:
!pip install BlackBoxAuditing

     |████████████████████████████████| 2.6 MB 4.7 MB/s 
  Created wheel for BlackBoxAuditing: filename=BlackBoxAuditing-0.1.54-py2.py3-none-any.whl size=1394771 sha256=2f18382574c5918b9cf98dfd8e6aeffb5db68bf60dfdc25a857a69a67a95bcfb
  Stored in directory: /root/.cache/pip/wheels/05/9f/ee/541a74be4cf5dad17430e64d3276370ea7b6a834a76cb4215a
Successfully built BlackBoxAuditing


#IMPORTS

In [6]:
import numpy as np
from mlxtend.feature_selection import  ExhaustiveFeatureSelector
from xgboost import  XGBClassifier
# import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import openpyxl
import xlsxwriter
from openpyxl import  load_workbook

import BlackBoxAuditing
import shap
#suppress setwith copy warning
pd.set_option('mode.chained_assignment',None)
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, SelectFwe, SelectPercentile,SelectFdr, SelectFpr, SelectFromModel
from sklearn.feature_selection import chi2, mutual_info_classif
# from skfeature.function.similarity_based import fisher_score
from aif360.algorithms.inprocessing import PrejudiceRemover, MetaFairClassifier, AdversarialDebiasing
import matplotlib.pyplot as plt
from aif360.metrics.classification_metric import ClassificationMetric

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing, LFR,OptimPreproc
from aif360.datasets import StandardDataset , BinaryLabelDataset
from sklearn.preprocessing import MinMaxScaler 
MM= MinMaxScaler()
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

import sys
sys.path.append("../")
import os



/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [7]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpwypdf6wc
  JVM stdout: /tmp/tmpwypdf6wc/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpwypdf6wc/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.6
H2O_cluster_version_age:,6 days
H2O_cluster_name:,H2O_from_python_unknownUser_ogr3vi
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


#**************************LOADING DATASET*******************************

In [8]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


# Meta



In [9]:
for i in range(1,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Violent/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path)
  first_column = train.pop('two_year_recid')
  train.insert(0, 'two_year_recid', first_column)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Violent/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path)
  first_column = test.pop('two_year_recid')
  test.insert(0, 'two_year_recid', first_column)


  # # normalization of train and test sets
  # Fitter= MM.fit(train)
  # transformed_train=Fitter.transform(train)
  # train=pd.DataFrame(transformed_train, columns= train.columns)

  # #test normalization
  # transformed_test=Fitter.transform(test)
  # test=pd.DataFrame(transformed_test, columns= test.columns)


  ## ****************CONVERTING TO BLD FORMAT******************************
  #BLD Train set
  class Train(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Train, self).__init__(df=train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )
  # BLD Test
  BLD_Train = Train(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  

  class Test(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  
  #******************************** MetaFair Classifier regularizer*****************************
  Classifier = MetaFairClassifier(tau=0, sensitive_attr= 'race')
  Classifier.fit(BLD_Train)
  prediction= Classifier.predict(BLD_Test)
  #**********************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS****************************
  #predicted labels
  # gbm_Predictions= best_model.predict(Test)
  # gbm_Predictions= gbm_Predictions.as_data_frame()
  predicted_df= test.copy()
  predicted_df['two_year_recid']= prediction.labels

  # ********************COMPUTE DISCRIMINATION*****************************

  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])


  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta0.xlsx')
  Violent= excelBook['Violent']
  data= Violent.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta0.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Violent', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

overflow encountered in exp
overflow encountered in add
overflow encountered in add
invalid value encountered in true_divide
invalid value encountered in true_divide
overflow encountered in add
invalid value encountered in true_divide
invalid value encountered in true_divide
invalid value encountered in true_divide


KeyboardInterrupt: ignored

# Meta 0.2



In [ ]:
for i in range(27,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Violent/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path)
  first_column = train.pop('two_year_recid')
  train.insert(0, 'two_year_recid', first_column)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Violent/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path)
  first_column = test.pop('two_year_recid')
  test.insert(0, 'two_year_recid', first_column)


  # # normalization of train and test sets
  # Fitter= MM.fit(train)
  # transformed_train=Fitter.transform(train)
  # train=pd.DataFrame(transformed_train, columns= train.columns)

  # #test normalization
  # transformed_test=Fitter.transform(test)
  # test=pd.DataFrame(transformed_test, columns= test.columns)


  ## ****************CONVERTING TO BLD FORMAT******************************
  #BLD Train set
  class Train(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Train, self).__init__(df=train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )
  # BLD Test
  BLD_Train = Train(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  

  class Test(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  
  #******************************** MetaFair Classifier regularizer*****************************
  Classifier = MetaFairClassifier(tau=0.2, sensitive_attr= 'race')
  Classifier.fit(BLD_Train)
  prediction= Classifier.predict(BLD_Test)
  #**********************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS****************************
  #predicted labels
  # gbm_Predictions= best_model.predict(Test)
  # gbm_Predictions= gbm_Predictions.as_data_frame()
  predicted_df= test.copy()
  predicted_df['two_year_recid']= prediction.labels

  # ********************COMPUTE DISCRIMINATION*****************************

  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])


  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta2.xlsx')
  Violent= excelBook['Violent']
  data= Violent.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta2.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Violent', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

# Meta 0.4



In [ ]:
for i in range(27,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Violent/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path)
  first_column = train.pop('two_year_recid')
  train.insert(0, 'two_year_recid', first_column)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Violent/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path)
  first_column = test.pop('two_year_recid')
  test.insert(0, 'two_year_recid', first_column)


  # # normalization of train and test sets
  # Fitter= MM.fit(train)
  # transformed_train=Fitter.transform(train)
  # train=pd.DataFrame(transformed_train, columns= train.columns)

  # #test normalization
  # transformed_test=Fitter.transform(test)
  # test=pd.DataFrame(transformed_test, columns= test.columns)


  ## ****************CONVERTING TO BLD FORMAT******************************
  #BLD Train set
  class Train(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Train, self).__init__(df=train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )
  # BLD Test
  BLD_Train = Train(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  

  class Test(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  
  #******************************** MetaFair Classifier regularizer*****************************
  Classifier = MetaFairClassifier(tau=0.4, sensitive_attr= 'race')
  Classifier.fit(BLD_Train)
  prediction= Classifier.predict(BLD_Test)
  #**********************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS****************************
  #predicted labels
  # gbm_Predictions= best_model.predict(Test)
  # gbm_Predictions= gbm_Predictions.as_data_frame()
  predicted_df= test.copy()
  predicted_df['two_year_recid']= prediction.labels

  # ********************COMPUTE DISCRIMINATION*****************************

  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])


  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta4.xlsx')
  Violent= excelBook['Violent']
  data= Violent.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta4.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Violent', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

In [ ]:
i

# Meta 0.6



In [ ]:
i

In [ ]:
for i in range(27,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Violent/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path)
  first_column = train.pop('two_year_recid')
  train.insert(0, 'two_year_recid', first_column)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Violent/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path)
  first_column = test.pop('two_year_recid')
  test.insert(0, 'two_year_recid', first_column)


  # # normalization of train and test sets
  # Fitter= MM.fit(train)
  # transformed_train=Fitter.transform(train)
  # train=pd.DataFrame(transformed_train, columns= train.columns)

  # #test normalization
  # transformed_test=Fitter.transform(test)
  # test=pd.DataFrame(transformed_test, columns= test.columns)


  ## ****************CONVERTING TO BLD FORMAT******************************
  #BLD Train set
  class Train(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Train, self).__init__(df=train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )
  # BLD Test
  BLD_Train = Train(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  

  class Test(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  
  #******************************** MetaFair Classifier regularizer*****************************
  Classifier = MetaFairClassifier(tau=0.6, sensitive_attr= 'race')
  Classifier.fit(BLD_Train)
  prediction= Classifier.predict(BLD_Test)
  #**********************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS****************************
  #predicted labels
  # gbm_Predictions= best_model.predict(Test)
  # gbm_Predictions= gbm_Predictions.as_data_frame()
  predicted_df= test.copy()
  predicted_df['two_year_recid']= prediction.labels

  # ********************COMPUTE DISCRIMINATION*****************************

  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])


  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta6.xlsx')
  Violent= excelBook['Violent']
  data= Violent.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta6.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Violent', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

# Meta 0.8



In [12]:
i

26

In [13]:
for i in range(27,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Violent/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path)
  first_column = train.pop('two_year_recid')
  train.insert(0, 'two_year_recid', first_column)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Violent/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path)
  first_column = test.pop('two_year_recid')
  test.insert(0, 'two_year_recid', first_column)


  # # normalization of train and test sets
  # Fitter= MM.fit(train)
  # transformed_train=Fitter.transform(train)
  # train=pd.DataFrame(transformed_train, columns= train.columns)

  # #test normalization
  # transformed_test=Fitter.transform(test)
  # test=pd.DataFrame(transformed_test, columns= test.columns)


  ## ****************CONVERTING TO BLD FORMAT******************************
  #BLD Train set
  class Train(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Train, self).__init__(df=train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )
  # BLD Test
  BLD_Train = Train(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  

  class Test(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  
  #******************************** MetaFair Classifier regularizer*****************************
  Classifier = MetaFairClassifier(tau=0.8, sensitive_attr= 'race')
  Classifier.fit(BLD_Train)
  prediction= Classifier.predict(BLD_Test)
  #**********************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS****************************
  #predicted labels
  # gbm_Predictions= best_model.predict(Test)
  # gbm_Predictions= gbm_Predictions.as_data_frame()
  predicted_df= test.copy()
  predicted_df['two_year_recid']= prediction.labels

  # ********************COMPUTE DISCRIMINATION*****************************

  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])


  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta8.xlsx')
  Violent= excelBook['Violent']
  data= Violent.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta8.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Violent', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

overflow encountered in exp
overflow encountered in add
invalid value encountered in true_divide
invalid value encountered in true_divide
overflow encountered in add
invalid value encountered in true_divide
invalid value encountered in true_divide
invalid value encountered in true_divide


Accuracy 0.7793017456359103


overflow encountered in exp
overflow encountered in add
overflow encountered in add
invalid value encountered in true_divide
overflow encountered in add
invalid value encountered in true_divide
overflow encountered in add
invalid value encountered in true_divide
invalid value encountered in true_divide
invalid value encountered in true_divide


Accuracy 0.7855361596009975


invalid value encountered in true_divide
invalid value encountered in true_divide
invalid value encountered in true_divide
invalid value encountered in true_divide
invalid value encountered in true_divide


...

Accuracy 0.6745635910224439


overflow encountered in exp
overflow encountered in add
overflow encountered in add
invalid value encountered in true_divide
overflow encountered in add
invalid value encountered in true_divide
overflow encountered in add
invalid value encountered in true_divide
invalid value encountered in true_divide
invalid value encountered in true_divide


Accuracy 0.7718204488778054


overflow encountered in exp
overflow encountered in add
overflow encountered in add
invalid value encountered in true_divide
overflow encountered in add
invalid value encountered in true_divide
overflow encountered in add
invalid value encountered in true_divide
invalid value encountered in true_divide
invalid value encountered in true_divide


Accuracy 0.756857855361596


In [14]:
i

50

# Meta 1.0

In [16]:
i

26

In [17]:
for i in range(27,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Violent/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path)
  first_column = train.pop('two_year_recid')
  train.insert(0, 'two_year_recid', first_column)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Violent/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path)
  first_column = test.pop('two_year_recid')
  test.insert(0, 'two_year_recid', first_column)


  # # normalization of train and test sets
  # Fitter= MM.fit(train)
  # transformed_train=Fitter.transform(train)
  # train=pd.DataFrame(transformed_train, columns= train.columns)

  # #test normalization
  # transformed_test=Fitter.transform(test)
  # test=pd.DataFrame(transformed_test, columns= test.columns)


  ## ****************CONVERTING TO BLD FORMAT******************************
  #BLD Train set
  class Train(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Train, self).__init__(df=train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )
  # BLD Test
  BLD_Train = Train(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  

  class Test(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  
  #******************************** MetaFair Classifier regularizer*****************************
  Classifier = MetaFairClassifier(tau=1.0, sensitive_attr= 'race')
  Classifier.fit(BLD_Train)
  prediction= Classifier.predict(BLD_Test)
  #**********************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS****************************
  #predicted labels
  # gbm_Predictions= best_model.predict(Test)
  # gbm_Predictions= gbm_Predictions.as_data_frame()
  predicted_df= test.copy()
  predicted_df['two_year_recid']= prediction.labels

  # ********************COMPUTE DISCRIMINATION*****************************

  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])


  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta10.xlsx')
  Violent= excelBook['Violent']
  data= Violent.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta10.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Violent', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

overflow encountered in exp
overflow encountered in add
invalid value encountered in true_divide
invalid value encountered in true_divide
overflow encountered in add
invalid value encountered in true_divide
invalid value encountered in true_divide
invalid value encountered in true_divide


Accuracy 0.8354114713216958


overflow encountered in exp
overflow encountered in add
overflow encountered in add
invalid value encountered in true_divide
overflow encountered in add
invalid value encountered in true_divide
overflow encountered in add
invalid value encountered in true_divide
invalid value encountered in true_divide
invalid value encountered in true_divide


Accuracy 0.7668329177057357


invalid value encountered in true_divide
invalid value encountered in true_divide
invalid value encountered in true_divide
invalid value encountered in true_divide
invalid value encountered in true_divide


...

Accuracy 0.683291770573566


overflow encountered in exp
overflow encountered in add
overflow encountered in add
invalid value encountered in true_divide
overflow encountered in add
invalid value encountered in true_divide
overflow encountered in add
invalid value encountered in true_divide
invalid value encountered in true_divide
invalid value encountered in true_divide


Accuracy 0.7743142144638404


overflow encountered in exp
overflow encountered in add
overflow encountered in add
invalid value encountered in true_divide
overflow encountered in add
invalid value encountered in true_divide
overflow encountered in add
invalid value encountered in true_divide
invalid value encountered in true_divide
invalid value encountered in true_divide


Accuracy 0.7817955112219451
